In [17]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
from dateutil.relativedelta import relativedelta

token = 't1.9euelZrNis_Hkp2dlMbMk56MnZCMle3rnpWamZmYkpuKnJWXip7KjJSTk4zl8_dnGlFP-e9GR14b_t3z9ydJTk_570ZHXhv-zef1656VmszIyZuKnM-Zk5GeipnIyseR7_zF656VmszIyZuKnM-Zk5GeipnIyseR.TcptA610g2krPfgxO7EiN1Ae8xhk8gTSRM1ylkjUaHD5iREf15eP2bRABo7GBfhCmNBByGTKVzkNdL0iY6DFBg'

# Обязательная инициирующую функция, в которой подгружается токен и которая используется для вызова других функций

headers={'Authorization':token,'Accept':'application/json'}


#'b1gb310irjlk6b99e14g' - аналитика
#'b1gc7vi2ckqausoc5dr7' - спутник

FOLDER_ID = 'b1gc7vi2ckqausoc5dr7' # id каталога из которого береться запрос
FOLDER = 'blocked_flats' #имя папки в которую будет выкладываться
ACCESS_KEY = 'YCAJEuIkha6my9B9Hb6W1GyNX' #aws_access_key_id для S3
SECRET_KEY = 'YCPJ6PS_Ll0vnFLuZEaNEXDKuB0J3vs2iSt5RWp6' #aws_secret_access_key в s3
BUCKET_NAME = 'aggregated-data' #имя бакета
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "temp_file"


def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    now = datetime.datetime.now(pytz.timezone(time_zone))    
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    last_month_data = now - relativedelta(months=1)
    return {'now':now.strftime('%Y-%m-%d %H:%M:%S'),'last_month_data':last_month_data.strftime('%Y-%m-%d'),'yesterday_data':yesterday.strftime('%Y-%m-%d'),'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 'year':yesterday.strftime('%Y'),'month':yesterday.strftime('%m'),'day':yesterday.strftime('%d')}

# Переменные, которые могут быть использованы в разоичных запросах. 
now = get_now_datetime_str()['now']
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
last_month_data = get_now_datetime_str()['last_month_data']

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах

# ниже записывается запрос
query_text = open('query.txt','r').read().format(last_month_data,yesterday_data)


def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'

def get_request(offset): # фунция возвращает ответ запроса. Максимум 1000 строк.
    offset = offset
    get_query_results_url = f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries/{request_id}/results/0?project={FOLDER_ID}&offset={str(offset)}&limit=1000'
    response = requests.get(
        get_query_results_url,
        headers = headers
    )
    return response




def if_cell_is_list(cell): # функция участвует в преобразовании данных при создании файла
    if isinstance(cell, list):
        if len(cell) == 0:
            return ''
        else: 
            return cell[0]
    else:
        return cell

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w')
    temp_file.write(special_str[:-1]+'\n')

    while response.status_code == 200 and len(response.json()['rows']) != 0:  #Цикл делает запросы по 10000, пока не кончатся данные
        response = get_request(offset)
        response_rows = response.json()['rows']
        rows = [[if_cell_is_list(cell) for cell in row] for row in response_rows]  #Преобразуются строки
        # Открывает созданный файл и добавляет в него строки
        for i in rows:
            special_str = ""
            for j in i:
                if isinstance(j, str):
                    special_str = f"{special_str}'{str(j)}',"
                else:
                    special_str = f"{special_str}{str(j)},"
            temp_file.write(special_str[:-1]+'\n')
        offset +=1000 # увеличивает смещение

def get_s3_instance(): # функция создает соединение
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )

def upload_dump_to_s3(): # функция выгружает данные в s3
    get_s3_instance().upload_file(
        Filename=TEMP_FILENAME,
        Bucket=BUCKET_NAME,
        Key=f"{FOLDER}/year={int(year)}/{int(month)}.csv"
    )

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

a=0
request_id = create_query()
while str(get_request(0)) == '<Response [400]>':
    time.sleep(10)
    a+=1

write_temp_file()
get_s3_instance()
upload_dump_to_s3()
remove_temp_files()

requests.get("https://healthchecks.sputnik.systems/ping/3e4f3767-81c9-48fa-9bb2-8ef8d15eba4e")



<Response [200]>

In [8]:
def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'

request_id = create_query()

In [13]:
request_id = create_query()

In [6]:
request_id

'csqufasv0uq4ovde89po'

In [14]:
get_request(0)

<Response [400]>

In [15]:
while str(get_request(0)) == '<Response [400]>':
    time.sleep(10) 